### Words.Persian: Hill cipher
### زُفر هیل
یکی از روشهای نهمب‌گذاری و نهمب‌گشایی، برپایه برابرشهای تانه ای است
<br>در این روش، کلید، یک ماتکدان چارسو است
<br> encryption=نهمب‌گذاری
<br> decryption=نهمب‌گشایی
###### by Hamed Shah-آHosseini
Explanation in Persian (پارسی) at Instagram: @words.persian
<br>https://www.instagram.com/words.persian/
<br>Code at: https://github.com/ostad-ai/Words-Persian

In [1]:
import numpy as np
import string

در اینجا، از الفبای انگلیسی بهره میگیریم

In [2]:
english_alphabet=string.ascii_lowercase
print('The characters of english alphabet:')
print(english_alphabet)

The characters of english alphabet:
abcdefghijklmnopqrstuvwxyz


هر نویسه (کاراکتر) در الفبای انگلیسی را با شماره ای دُریست (اینتیجر)، برپایه پیچازه (تیبل) زیر، جایگزین میکنیم

In [3]:
# the codes from letter to integer
print('alphabet table:\n')
for char in english_alphabet:
    print(char,':',english_alphabet.find(char),end=' ,')

alphabet table:

a : 0 ,b : 1 ,c : 2 ,d : 3 ,e : 4 ,f : 5 ,g : 6 ,h : 7 ,i : 8 ,j : 9 ,k : 10 ,l : 11 ,m : 12 ,n : 13 ,o : 14 ,p : 15 ,q : 16 ,r : 17 ,s : 18 ,t : 19 ,u : 20 ,v : 21 ,w : 22 ,x : 23 ,y : 24 ,z : 25 ,

دیدیم که چگونه وارونه چندینکردی را با الگوریتم فراگسترده اقلیدسی، انجام 
دهیم
<br>در اینجا، الگوریتم فراگسترده اقلیدسی را به روز رسانی کرده تا برای شماره های ناینده (نگاتیو) نیز، به درستی کار کند

In [4]:
# version 2: for handling negative integers too
# Extended Euclidean algorithm
# returns GCD, and bezout parameters s and t
def extended_Euc_v2(a0,b0):
    a,b= abs(a0),abs(b0)
    swapFlag=False
    if a<b:
        a,b=b,a
        swapFlag=True
    r_old,r_new=a,b
    s_old,s_new=1,0
    t_old,t_new=0,1
    while r_new>0:
        q=r_old//r_new
        r_old,r_new=r_new,r_old-q*r_new
        s_old,s_new=s_new,s_old-q*s_new
        t_old,t_new=t_new,t_old-q*t_new    
    if swapFlag:
        if a0<0: t_old=-t_old
        if b0<0: s_old=-s_old
        return r_old,t_old,s_old
    else:
        if a0<0: s_old=-s_old
        if b0<0: t_old=-t_old
        return r_old,s_old,t_old
    
# multiplicative inverse of a with modulus m
# which handles both engative and pisitve a
def mult_inverse(a,m):
    gcd,s,t=extended_Euc_v2(a,m)
    if gcd!=1:
        return None
    else:
        return s

بدست آوردن بریهنده (دترمینان) و همساختگر (کوفاکتور) ماتکدان (ماتریس) های سه در سه

In [7]:
# Computing determinant and cofactor matrix
# for 3-by-3 matrices

#determinant for a 3*3 matrix
def det_3(M):
    a=M[0,0]; b=M[0,1]; c=M[0,2]
    d=M[1,0]; e=M[1,1]; f=M[1,2]
    g=M[2,0]; h=M[2,1]; i=M[2,2]
    A=e*i-f*h; B=-(d*i-f*g); C=d*h-e*g
    return a*A+b*B+c*C

#cofactor matrix for a 3*3 matrix
def cofactor_3(M):
    a=M[0,0]; b=M[0,1]; c=M[0,2]
    d=M[1,0]; e=M[1,1]; f=M[1,2]
    g=M[2,0]; h=M[2,1]; i=M[2,2]
    A=e*i-f*h; B=-(d*i-f*g); C=d*h-e*g
    D=-(b*i-c*h); E=a*i-c*g; F=-(a*h-b*g)
    G=b*f-c*e; H=-(a*f-c*d); I=a*e-b*d
    detM=a*A+b*B+c*C
    if detM==0: return None
    M_cf=np.zeros_like(M)
    M_cf[0,0]=A; M_cf[0,1]=B; M_cf[0,2]=C
    M_cf[1,0]=D; M_cf[1,1]=E; M_cf[1,2]=F
    M_cf[2,0]=G; M_cf[2,1]=H; M_cf[2,2]=I
    return M_cf

الگوریتم نهمب‌گذاری و نهمب‌گشایی برای روش زُفر هیل

In [8]:
# Hill cipher by 3*3 matrix as key
class Hill_cipher_3:
    def __init__(self,alphabet,H=None):
        self.m=len(alphabet)
        self.n=3
        self.alphabet=alphabet
        if H is None:
            flag=True
            while flag:
                H=np.random.randint(0,self.m,(self.n,self.n))
                detH=int(det_3(H))
                GCD,_,_=extended_Euc_v2(detH,self.m)
                if (abs(detH)>0) and (GCD==1):
                    flag=False        
        self.H=H
        detH=int(det_3(self.H))
        detH_inv=mult_inverse(detH,self.m)%self.m
        cofactor=cofactor_3(self.H)
        self.H_inv=detH_inv*cofactor.T
        self.H_inv%=self.m
        self.H_inv=np.round(self.H_inv)
    def encrypt(self,msg):
        p_text=np.zeros(self.n,dtype='int')
        msg_enc=''
        counter=0
        rem=len(''.join(msg.split()))%self.n
        if rem!=0:
            rem=self.n-rem
            while rem>0:
                msg+=self.alphabet[np.random.randint(0,self.m)]
                rem-=1
        for word in msg.split(' '):            
            for letter in word:
                index=self.alphabet.find(letter)
                p_text[counter]=index
                counter+=1
                if counter==self.n:
                    counter=0
                    c_text=np.dot(self.H,p_text.reshape(-1,1)).ravel()
                    c_text%=self.m
                    for i in range(self.n):
                        msg_enc+=self.alphabet[c_text[i]]   
        return msg_enc
    def decrypt(self,msg_enc):
        c_text=np.zeros(self.n,dtype='int')
        counter=0
        msg_dec=''
        for letter in msg_enc:
            index=self.alphabet.find(letter)
            c_text[counter]=index
            counter+=1
            if counter==self.n:
                counter=0
                p_text=np.dot(self.H_inv,c_text.reshape(-1,1)).ravel()
                p_text%=self.m
                for i in range(self.n):
                    msg_dec+=self.alphabet[int(p_text[i])]  
        return msg_dec

درست کردن نمونه ای از روی دسته زُفر هیل

In [9]:
# An instance of the class Hill_cipher_3
hcipher=Hill_cipher_3(english_alphabet)

نمونه ای از یک پیغام، که نخست، نهمب‌گذاری میشود، و سپس نهمب‌گشایی میشود

In [13]:
message='hello world from hill cipher'
encrypted=hcipher.encrypt(message)
decrypted=hcipher.decrypt(encrypted)
print(f'plaintext: {message}')
print(f'ciphertext: {encrypted}')
print(f'decrypted text: {decrypted}')

plaintext: hello world from hill cipher
ciphertext: scpnvytqhyrxbfblubfxtygh
decrypted text: helloworldfromhillcipher
